<a href="https://colab.research.google.com/github/fpseverino/progetto-ml-ai/blob/main/Severino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import copy
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, HalvingGridSearchCV, cross_val_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

# Classe Task
Viene definita una classe `Task`, il cui inizializzatore prende come parametro un `DataFrame` e ha come attributi le suddivisioni delle feature e delle label per il training e il testing, che saranno uguali per tutti i Task.
L'inizializzatore accetta anche un parametro booleano che attiva o meno la feature selection con `SelectKBest`.

La classe contiene anche un metodo per scalare le proprie feature con lo scaler passato come argomento.

In [2]:
class Task:
  def __init__(self, df, select = False):
    X = df.drop("Label", axis = 1)
    X = X.drop("Task", axis = 1)
    X = X.drop("Id", axis = 1)
    y = df["Label"]

    if select:
      X = SelectKBest(f_classif, k = 100).fit_transform(X, y)

    # Passando un `random_state` a `train_test_split`, in tutti i Task verranno selezionati gli stessi sample
    self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

  def scale(self, sc):
    sc.fit(self.X_train)
    self.X_train = sc.transform(self.X_train)
    self.X_test = sc.transform(self.X_test)

## Array `tasks`
Vengono letti i `csv` dei Task già ripuliti e vengono salvati come oggetti `Task`, effettuando la feature selection, all'interno dell'array `tasks`.

In [3]:
tasks = []
for i in range(11, 22):
  df = pd.read_csv("Task_{}_clean.csv".format(i))
  task = Task(df, select = True)
  tasks.append(task)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [2] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [8] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [2] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/py

## Voto finale rispetto i Task
Per valutare il voto finale che ogni modello attribuirà a ogni sample del set di testing, viene calcolata la moda delle lable per ogni sample attraverso tutti i set di testing (che contengono tutti gli stessi sample, vedi la classe `Task`), e viene definita la funzione `similarity` per calcolarne la somiglianza con i risultati delle predizioni.

In [4]:
y_tests = []
for task in tasks:
  y_tests.append(copy.deepcopy(task.y_test))

y_test_mode, y_test_count = stats.mode(y_tests)

def similarity(predictions_mode):
  if len(y_test_mode) != len(predictions_mode):
    return 0.0

  correct = 0
  for i in range(len(predictions_mode)):
    if y_test_mode[i] == predictions_mode[i]:
      correct += 1

  return "{:.2%} ({}/{})".format(correct / len(predictions_mode), correct, len(predictions_mode))

# Classificazione Boosting
Nel Boosting, più modelli vengono generati consecutivamente dando sempre più peso agli errori effettuati nei modelli precedenti. In questo modo si creano modelli via via più "attenti" agli aspetti che hanno causato inesattezze nei modelli precedenti, ottenendo infine un modello aggregato avente migliore accuratezza di ciascun modello che lo costituisce.

## Funzione `test_boosting`
Viene definita una funzione `test_boosting` che esegue l'allenamento dei modelli di boosting e ne stampa le metriche risultanti.

Dopo aver eventualmente scalato i `Task` mediante uno scaler, viene eseguita, task per task, la cross validation, poi un fit del classificatore, e vengono salvati gli score e le predizioni. Infine verrà stampata la media fra tutti i Task degli score, e la moda delle predizioni sul test set paragonata a quella effettiva.

In [5]:
def test_boosting(clf, sc = None):
  test_tasks = copy.deepcopy(tasks)

  if sc is not None:
    for task in test_tasks:
      task.scale(sc)

  cross_val_scores = []
  scores = []
  predictions = []

  for task in test_tasks:
    cross_val_scores.append(cross_val_score(clf, task.X_train, task.y_train, cv = 4, n_jobs = -1).mean())
    clf.fit(task.X_train, task.y_train)
    scores.append(clf.score(task.X_test, task.y_test))
    predictions.append(clf.predict(task.X_test))

  print("Results{}:".format("" if sc is None else " (scaled tasks)"))
  print(" - Mean cross_val_score: {:.2%}".format(np.mean(cross_val_scores)))
  print(" - Mean score:           {:.2%}".format(np.mean(scores)))

  mode, count = stats.mode(predictions)
  print(" - Similarity:           {}\n".format(similarity(mode)))

## Classificatore `AdaBoost`
Un classificatore AdaBoost è un meta-stimatore che inizia inserendo un classificatore nel set di dati originale e quindi adatta copie aggiuntive del classificatore nello stesso set di dati, ma dove i pesi delle istanze classificate in maniera errata vengono regolati in modo tale che i classificatori successivi si concentrino maggiormente sui casi difficili.

Viene effettuata una prova di ottimizzazione del classificatore `AdaBoost` senza scaling e una con uno scaler `MinMax`. Tendenzialmente si ottengono risultati leggermente migliori se le feature vengono scalate.

In [12]:
clf = AdaBoostClassifier(n_estimators = 100, algorithm = "SAMME", random_state = 0)
test_boosting(clf)
test_boosting(clf, MinMaxScaler(feature_range = (0, 1)))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


Results:
 - Mean cross_val_score: 83.14%
 - Mean score:           86.36%
 - Similarity:           91.67% (22/24)



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


Results (scaled tasks):
 - Mean cross_val_score: 86.12%
 - Mean score:           89.39%
 - Similarity:           91.67% (22/24)



## Classificatore Gradient Boosting

Questo algoritmo costruisce un modello additivo in modo graduale; consente l'ottimizzazione di funzioni di perdita differenziabili arbitrarie. In ogni fase, `n_classes_` alberi di regressione sono adattati al gradiente negativo della funzione di perdita, ad es. perdita di log binaria o multiclasse.

Viene effettuata una prova di ottimizzazione del classificatore Gradient Boosting senza scaling e una con uno scaler `MinMax`. Tendenzialmente, in entrambe le prove i risultati sono simili.

In [7]:
clf = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.01, random_state = 0)
test_boosting(clf)
test_boosting(clf, MinMaxScaler(feature_range = (0, 1)))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


Results:
 - Mean cross_val_score: 83.75%
 - Mean score:           89.02%
 - Similarity:           91.67% (22/24)



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


Results (scaled tasks):
 - Mean cross_val_score: 86.06%
 - Mean score:           87.12%
 - Similarity:           91.67% (22/24)



## Classificatore `HistGradientBoosting`
Durante l'addestramento, il coltivatore di alberi impara ad ogni punto di divisione se i sample con valori mancanti devono andare al figlio sinistro o destro, in base al potenziale guadagno. Durante la previsione, i sample con valori mancanti vengono assegnati di conseguenza al figlio sinistro o destro. Se durante l'addestramento non sono stati rilevati valori mancanti per una determinata feature, i sample con valori mancanti vengono mappati a qualunque figlio abbia il maggior numero di sample.

Viene effettuata una prova di ottimizzazione del classificatore `HistGradientBoosting` senza scaling e una con uno scaler `MinMax`. Tendenzialmente, in entrambe le prove i risultati sono identici.

In [13]:
clf = HistGradientBoostingClassifier(max_iter = 500, learning_rate = 0.01, random_state = 0)
test_boosting(clf)
test_boosting(clf, MinMaxScaler(feature_range = (0, 1)))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


Results:
 - Mean cross_val_score: 83.80%
 - Mean score:           89.77%
 - Similarity:           95.83% (23/24)



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


Results (scaled tasks):
 - Mean cross_val_score: 83.80%
 - Mean score:           89.77%
 - Similarity:           95.83% (23/24)



# Classificatore Percettrone multi-strato
Vengono scalati i task tramite uno `StandardScaler`, in seguito viene eseguita una ricerca sulla griglia di parametri con un `HalvingGridSearchCV` (versione più veloce di `GridSearchCV`), e infine viene testato su tutti i Task un `MLPCLassifier`, in modo simile a come sono stati testati i modelli di boosting, inizializzandolo con i migliori iperparametri trovati.

In [14]:
sc = StandardScaler()

scaled_tasks = copy.deepcopy(tasks)

for task in scaled_tasks:
  task.scale(sc)

param_grid = {
  "hidden_layer_sizes": [(150, 100, 50), (120, 80, 40), (100, 50, 30)],
  "activation": ['identity', 'logistic', 'tanh', 'relu'],
  "alpha": 10.0 ** -np.arange(1, 7),
  "max_iter": [1000, 2000, 3000]
}
search = HalvingGridSearchCV(MLPClassifier(solver = "lbfgs", random_state = 0), param_grid, cv = 4, n_jobs = -1)

cross_val_scores = []
scores = []
predictions = []

for task in scaled_tasks:
  search.fit(task.X_train, task.y_train)

  clf = MLPClassifier(**search.best_params_, solver = "lbfgs", random_state = 0)

  cross_val_scores.append(cross_val_score(clf, task.X_train, task.y_train, cv = 4, n_jobs = -1).mean())
  clf.fit(task.X_train, task.y_train)
  scores.append(clf.score(task.X_test, task.y_test))
  predictions.append(clf.predict(task.X_test))

print("Results (scaled tasks):")
print(" - Mean cross_val_score: {:.2%}".format(np.mean(cross_val_scores)))
print(" - Mean score:           {:.2%}".format(np.mean(scores)))

mode, count = stats.mode(predictions)
print(" - Similarity:           {}\n".format(similarity(mode)))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklea

Results (scaled tasks):
 - Mean cross_val_score: 75.98%
 - Mean score:           80.30%
 - Similarity:           79.17% (19/24)

